In [ ]:
import pandas as pd

In [ ]:
def prepData(csv):
    
    df_f = pd.DataFrame()
    for x in csv:
        df = pd.read_csv(x)
        df =  (df[df['AreaName'].isin(['France'])]
                    .set_index('Year')['Value'])
        df_f = pd.concat([df_f, df], axis = 1)
    
    df_f.columns = ["price", 'supply']
    df_f = df_f.dropna()
    
    return df_f

In [ ]:
#test = pd.read_csv('146908e8-7a8a-40ea-b670-b39daab67a15.csv')
#(test[test['AreaName'].isin(['France'])]
#                    .set_index('Year')['Value'])

In [ ]:
csv = ['146908e8-7a8a-40ea-b670-b39daab67a15.csv',
      'b24e9b0e-4b97-4acc-90f8-599cc178d434.csv']
df = prepData(csv = csv)

In [ ]:
df.head()

In [ ]:
axw = df.loc[:,['price','supply']].plot(secondary_y = 'price',title='wheat price and supply in levels')
figw = axw.get_figure()

## Returns, ADF test

1. Compute the first difference for the price and supply of wheat in France
2. Perform Augmented Dickey-Fuller tests on both level and first difference5
. Note that from a
plot, it is not clear that the level have a unit root, especially the supply seems to be erratic6
. If
time allow, you can check the ADF test with or without trend

In [ ]:
from statsmodels.tsa.stattools import adfuller
import pandas as pd
def adf_test(y):
    # perform Augmented Dickey Fuller test
    print('Results of Augmented Dickey-Fuller test:')
    dftest = adfuller(y, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['test statistic', 'p-value', '# of lags', '# of observations'])
    for key, value in dftest[4].items():
        dfoutput['Critical Value ({})'.format(key)] = value
    print(dfoutput)

In [ ]:
adf_test(y = df['price'].shift(1).dropna())

In [ ]:
adf_test(y = df['supply'].shift(1).dropna())

## 4 VAR

1. Select the order of a VAR with the two first difference
2. Fit the VAR with the selected order
3. Plot impulse response functions

The VAR class assumes that the passed time series are stationary. Non-stationary or trending data can often be transformed to be stationary by first-differencing or some other method. For direct analysis of non-stationary time series, a standard stable VAR(p) model is not appropriate.

In [ ]:
df_diff = df.pct_change(1).dropna()
df_diff.head()

In [ ]:
from statsmodels.tsa.api import VAR

In [ ]:
varmodel = VAR(df_diff)


In [ ]:
print(varmodel.select_order(maxlags = 5,
                            trend = 'c')
     )
### 1

In [ ]:
results = varmodel.fit(1)
results.summary()

In [ ]:
results.irf().plot()

## Correlation

1. Add the lagged variables of both the price and the supply
2. Compute the correlations between the variables: price, supply, lagged price and lagged supply

In [ ]:
df['priceLag'] = df['price'].shift(1)
df['supplyLag'] = df['supply'].shift(1)

In [ ]:
df.corr()